# **Repaying Debt**

#### Repaying debt is a straightforward process. The borrower must pay the exact amount owed plus the protocol usage fee. When a position is already below the liquidation threshold at the time of processing, the position will be liquidated and any pending request for repayment may be refunded.

## Key Protocol Parameters:

- **Liquidation Threshold:** Determines the threshold where the liquidation is triggered.
- **Protocol Usage Fee:** A fee charged for using the Halalend Protocol to create a loan position.

## On-chain Validation:

- **Liquidity Pool UTXO:** Represents the available liquidity for a given asset, this will be consumed and create a new UTXO where the loan repayment amount is returned to this pool.
- **Lending Validator:** A smart contract on Cardano that validates repayment requests and ensures they comply with protocol rules.
- **Protocol Parameters UTXO:** Stores the protocol's operational parameters and lending criteria for reference in transactions.
- **Oracle/DEX Liquidity Pool UTXO:** Provides current exchange rates for collateral and loan assets, essential for accurate valuation and risk assessment.
- **Collateral Validator:** A smart contract on Cardano that validates if the collateral utxo can be unlocked.

## Off-chain Infrastructure:

- **Batcher:** An essential background service that batches transactions for processing. This component is crucial due to Cardano's UTXO model, where a UTXO can only be consumed by one transaction in a block. The batcher ensures multiple transactions can be processed efficiently.

### Assuming the following Protocol Parameters:

In [22]:
let LIQUIDATION_THRESHOLD = 1.2
let PROTOCOL_USAGE_FEE = 5.0

### And the following calculations:

##### *Note: these are simplified calculations*

In [23]:
open System.Collections.Generic

// Define the type for easier handling
type CurrencyAmount = (string * (string * float) list)

// Calculates the current collateral ratio of a loan position given the exchange rates of the collateral and the loan
let calc_collateral_ratio total_collateral_value_in_usd total_loan_amount_in_usd = total_collateral_value_in_usd / total_loan_amount_in_usd

// Function to extract amount value from the list inside the tuple
let get_amount currency (amounts: CurrencyAmount list) =
    amounts
    |> List.tryFind (fun (c, _) -> c = currency) // Find the tuple with the matching currency
    |> Option.map snd // Get the second part of the tuple, which is the list
    |> Option.map (List.tryFind (fun (key, _) -> key = "amount")) // Find the amount in the list
    |> Option.bind id // Flatten the Option<Option<_>> to Option<_>
    |> Option.map snd // Get the amount value

// Function to compare loan and repayment amounts for a given currency
let compare_amount currency (loans: CurrencyAmount list) (repayments: CurrencyAmount list) =
    match get_amount currency loans, get_amount currency repayments with
    | Some loan_amount, Some repayment_amount -> repayment_amount >= loan_amount
    | _ -> false // If either amount is not found, we cannot say repayment is sufficient
    
    

### Now we need to define the loan parameters and current exchange rates

Assuming the loan position has not reached the loan term expiration and is not yet eligibile for liquidation. In an actual Cardano Transaction, the NFT identifier for the loan position must be present in the transaction as this proves ownership of the collateral. For simplicity's sake, we will not be putting the NFT in this simulation.

In [24]:

// Sample data
let loan_amount: CurrencyAmount list = [("ADA", [("amount", 1000.0)])]
let repayment_amount: CurrencyAmount list = [("ADA", [("amount", 1000.0)])]

// These are the collaterals sent by the borrower to the contract
let locked_collateral_assets = 
    [("iUSD", dict [("amount", box 1000.0)])
     ("halalend", dict [("amount", box 1000.0)])
     ("hosky", dict [("amount", box 100000.0)])]
    |> Map.ofList

// Test the function
let isRepaymentSufficient = compare_amount "ADA" loan_amount repayment_amount

if isRepaymentSufficient then
    printf "Repayment is sufficient\n"
    printf "Collateral is unlocked and sent back to the borrower\n"
    printf "Protocol usage fee collected %f\n" PROTOCOL_USAGE_FEE
    printf "Collateral assets unlocked %A\n" locked_collateral_assets
else
    printf "Repayment is not sufficient"





Repayment is sufficient
Collateral is unlocked and sent back to the borrower
Protocol usage fee collected 5.000000
Collateral assets unlocked map
  [("halalend", seq [[amount, 1000]]); ("hosky", seq [[amount, 100000]]);
   ("iUSD", seq [[amount, 1000]])]
